In [1]:
from CakePDFEmbedder import VectorDBLoader
from CakePDFEmbedder import VectorDBRetriever
from CakePDFEmbedder import ProcessPDF
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from langchain_openai import AzureChatOpenAI

In [3]:
import os
import subprocess
import base64
!export AZURE_API_VERSION="`kubectl get secret azure-creds -n kubeflow-brad -o jsonpath='{.data.AZURE_API_VERSION}' | base64 --decode`"
!export AZURE_OPENAI_API_KEY="`kubectl get secret azure-creds -n kubeflow-brad -o jsonpath='{.data.AZURE_OPENAI_API_KEY}' | base64 --decode`"
!export AZURE_OPENAI_ENDPOINT="`kubectl get secret azure-creds -n kubeflow-brad -o jsonpath='{.data.AZURE_OPENAI_ENDPOINT}' | base64 --decode`"
def kubectl_get_secret(secret_name, key):
    # Executing kubectl command to get the secret and extracting the value of the specified key
    command = f"kubectl get secret {secret_name} -n kubeflow-brad -o jsonpath={{.data.{key}}}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    # Checking if the command was successful
    if result.returncode == 0:
        # Decoding the base64-encoded value and return
        decoded_value = base64.b64decode(result.stdout.strip()).decode('utf-8')
        return decoded_value

# Fetching values from Kubernetes secret and decode
AZURE_API_VERSION = kubectl_get_secret("azure-creds", "AZURE_API_VERSION")
AZURE_OPENAI_API_KEY = kubectl_get_secret("azure-creds", "AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = kubectl_get_secret("azure-creds", "AZURE_OPENAI_ENDPOINT")

# Setting the environment variables
os.environ["AZURE_API_VERSION"] = AZURE_API_VERSION
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT

In [4]:
llm = AzureChatOpenAI(
    openai_api_version=os.getenv("AZURE_API_VERSION"),
    azure_deployment="gpt-4o",
    temperature=0
)

In [7]:
vector_store = MilvusVectorStore(uri="http://stepstone-milvus.milvus.svc.cluster.local:19530", dim=1024, overwrite=False, collection_name='rene_319_load_v2')

In [6]:
embedding_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3",
    max_length=4096
)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
myProcessor = ProcessPDF(llm)

In [ ]:
myBlock_dicts = myProcessor.partition_file_via_open_source('./rene_test/'+"2023.12.31.PortSum.Vista V. 2.pdf")

In [ ]:
myAugmentedDF = myProcessor.create_dataframe(myBlock_dicts)

In [ ]:
blocks_of_text = myProcessor.create_block_text(myAugmentedDF)

In [ ]:
myLoader = VectorDBLoader(vector_store, embedding_model)

In [ ]:
myLoader.addToVectorDB(blocks_of_text)

In [40]:
import pandas as pd
from pymilvus import connections, utility, MilvusClient
import pprint
import random
import re

In [27]:
analysts_df = pd.read_csv("./2024_Q1_PDF/Analysts Results/Quarterly Report data extraction.csv", encoding = "ISO-8859-1")
analysts_df.head(3)

,Assignment,File Name,URL,Sector,Fund Name,Asset_Name,entry_date,Unrealized Val,"Revenue (LTM), Unit as is in the report",Unit,Date (as is in the report),EBITDA (LTM),Unit.1,Date,Net Debt,Unit.2,Date.1,Note,Unnamed: 18
0,Example,2024.03.31.Ltr.American VI..pdf,Z:\01 - Research\Portfolio Management\Generati...,Large Buyout,"American Securities Partners VI, L.P.","Learning Care Group (US), Inc.",5/15/2014,"1,368,300,000","$1,566.80",Million,3/31/2024,$192.30,Million,3/31/2024,$838.40,Million,3/31/2024,NaN,NaN
1,AJ,2024.03.31.Ltr.American VI..pdf,Z:\01 - Research\Portfolio Management\Generati...,Large Buyout,"American Securities Partners VI, L.P.","Learning Care Group (US), Inc.",5/15/2014,"1,368,300,000","$1,566.80",Million,3/31/2024,$192.30,Million,3/31/2024,$838.40,Million,3/31/2024,NaN,NaN
2,AJ,2024.03.31.Ltr.Bain X..pdf,Z:\01 - Research\Portfolio Management\Generati...,Global Buyout,"Bain Capital Fund X, L.P.","Bob's Discount Furniture, Inc.",2/14/2014,"792,189,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
abbr_l = ['File Name', 'Fund Name', 'Asset_Name', 'Revenue (LTM), Unit as is in the report', 'EBITDA (LTM)', 'Net Debt']
abbr_analysts_df = analysts_df[abbr_l]
abbr_analysts_df['Revenue (LTM)']=abbr_analysts_df['Revenue (LTM), Unit as is in the report']
abbr_analysts_df.drop(columns=['Revenue (LTM), Unit as is in the report'], inplace=True)
abbr_analysts_df.head(3)

/tmp/ipykernel_71549/2738223046.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abbr_analysts_df['Revenue (LTM)']=abbr_analysts_df['Revenue (LTM), Unit as is in the report']
/tmp/ipykernel_71549/2738223046.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abbr_analysts_df.drop(columns=['Revenue (LTM), Unit as is in the report'], inplace=True)


,File Name,Fund Name,Asset_Name,EBITDA (LTM),Net Debt,Revenue (LTM)
0,2024.03.31.Ltr.American VI..pdf,"American Securities Partners VI, L.P.","Learning Care Group (US), Inc.",$192.30,$838.40,"$1,566.80"
1,2024.03.31.Ltr.American VI..pdf,"American Securities Partners VI, L.P.","Learning Care Group (US), Inc.",$192.30,$838.40,"$1,566.80"
2,2024.03.31.Ltr.Bain X..pdf,"Bain Capital Fund X, L.P.","Bob's Discount Furniture, Inc.",NaN,NaN,NaN


In [16]:
uri="http://stepstone-milvus.milvus.svc.cluster.local:19530"
dim=1024
collection_name='rene_319_load_v2'

In [17]:
# Use no-schema Milvus client.
mc = MilvusClient(uri=uri)

In [19]:
# pprint.pprint(mc.describe_collection(collection_name))

In [ ]:
ofile="2024.03.31.Ltr.American VI..pdf"
doc=ofile.lower().replace("..",".")
results = mc.search(
    collection_name,
    data=query_vectors,
    # filter='$meta["GPID"] in [4411]')
    filter=f'$meta["Document Name"] in ["{doc}"]',
    limit=50
    )

In [44]:
%%time
abbr_analysts_df['Model - Revenue (LTM)'] = abbr_analysts_df['Asset_Name'].apply(lambda x: str(query_engine.query(f"What is {x} most recent Net Revenue (LTM)?")))

CPU times: user 19.8 s, sys: 318 ms, total: 20.1 s
Wall time: 9min 35s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [86]:
abbr_analysts_df['Model - Revenue (LTM) - cleaned'] = abbr_analysts_df['Model - Revenue (LTM)'].apply(lambda x: reg_cleaner(x))

/tmp/ipykernel_71549/1900154242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abbr_analysts_df['Model - Revenue (LTM) - cleaned'] = abbr_analysts_df['Model - Revenue (LTM)'].apply(lambda x: reg_cleaner(x))


In [92]:
revenues = ['File Name', 'Asset_Name', 'Revenue (LTM)', 'Model - Revenue (LTM) - cleaned']
ebitdas = ['File Name', 'Asset_Name', 'EBITDA (LTM)', 'Model - EBITDA (LTM) - cleaned']
netdebts = ['File Name', 'Asset_Name', 'Net Debt', 'Model - Net Debt - cleaned']
abbr_analysts_df[netdebts]

,File Name,Asset_Name,Net Debt,Model - Net Debt - cleaned
0,2024.03.31.Ltr.American VI..pdf,"Learning Care Group (US), Inc.",$838.40,$736.3
1,2024.03.31.Ltr.American VI..pdf,"Learning Care Group (US), Inc.",$838.40,$736.3
2,2024.03.31.Ltr.Bain X..pdf,"Bob's Discount Furniture, Inc.",NaN,$0.7
3,2024.03.31.Ltr.Bain X..pdf,Canada Goose,NaN,
4,2024.03.31.Rpt.Fortress V..pdf,CW Financial Services,"$147,960.00",$147960
...,...,...,...,...
503,2024.03.31.Ltr.WestView III..pdf,Jopari Solutions,NaN,2685.2
504,2024.03.31.Rpt.Align III..pdf,Counsel Press Inc.,NaN,$53.0
505,2024.03.31.Rpt.Align III..pdf,Global Guardian LLC,NaN,0.9
506,2024.03.31.Rpt.Rubicon Tech III..pdf,BrandMaker,NaN,$3066


In [93]:
revenue_df = abbr_analysts_df[abbr_analysts_df['Revenue (LTM)'].notna()][revenues]
ebitda_df  = abbr_analysts_df[abbr_analysts_df['EBITDA (LTM)'].notna()][ebitdas]
netdebt_df = abbr_analysts_df[abbr_analysts_df['Net Debt'].notna()][netdebts]

In [73]:
!pwd

/home/jovyan/shared/projects/sample-notebooks/quarterly_reports_project/Grant Kim


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [95]:
revenue_df.to_csv("./2024_Q1_PDF/Analysts Results/Revenue.csv", index=False)
ebitda_df.to_csv("./2024_Q1_PDF/Analysts Results/EBITDA.csv", index=False)
netdebt_df.to_csv("./2024_Q1_PDF/Analysts Results/NetDebt.csv", index=False)
abbr_analysts_df.to_csv("./2024_Q1_PDF/Analysts Results/FullReport.csv", index=False)

In [51]:
%%time
abbr_analysts_df['Model - EBITDA (LTM)'] = abbr_analysts_df['Asset_Name'].apply(lambda x: str(query_engine.query(f"What is {x} most recent EBITDA (LTM)?")))

CPU times: user 19.7 s, sys: 355 ms, total: 20.1 s
Wall time: 9min 26s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [88]:
abbr_analysts_df['Model - EBITDA (LTM) - cleaned'] = abbr_analysts_df['Model - EBITDA (LTM)'].apply(lambda x: reg_cleaner(x))

/tmp/ipykernel_71549/3751406824.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abbr_analysts_df['Model - EBITDA (LTM) - cleaned'] = abbr_analysts_df['Model - EBITDA (LTM)'].apply(lambda x: reg_cleaner(x))


In [65]:
%%time
abbr_analysts_df['Model - Net Debt'] = abbr_analysts_df['Asset_Name'].apply(lambda x: str(query_engine.query(f"What is {x}'s most recent Net Debt?")))

CPU times: user 18.9 s, sys: 388 ms, total: 19.3 s
Wall time: 8min 20s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [91]:
abbr_analysts_df['Model - Net Debt - cleaned'] = abbr_analysts_df['Model - Net Debt'].apply(lambda x: reg_cleaner(x))

/tmp/ipykernel_71549/969140253.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abbr_analysts_df['Model - Net Debt - cleaned'] = abbr_analysts_df['Model - Net Debt'].apply(lambda x: reg_cleaner(x))


In [84]:
def reg_cleaner(reg_str):
    temp = re.sub("[^0-9$.]", "", reg_str)
    temp = re.sub("[.]$", "", temp)
    return re.sub("^[.]", "", temp)

In [83]:
re.sub("[^0-9$.]", "", "Something. $34,567.08.")

'.$34567.08.'

In [85]:
reg_cleaner("Something. $34,567.08.")

'$34567.08'

In [8]:
myRetriever = VectorDBRetriever(vector_store, embedding_model, "default", 2)

In [9]:
query_engine = RetrieverQueryEngine.from_args(myRetriever)

In [25]:
query_str = "What is AdSwerve's most recent Net Revenue (LTM)?"
response = query_engine.query(query_str)
print(str(response))

AdSwerve's most recent Net Revenue (LTM) is $78,605.
